In [54]:
"""
Test of manually building a low-rank density matrix that reproduces energy of (H_2)_2
"""

from pyscf import gto, scf
import numpy as np

# Build two H3 molecules far apart (say 10 Angstroms separation)
mol = gto.Mole()
mol.atom = '''
H 0 0 0
H 0.74 0 0.
H 9 0 0
H 9.74 0 0
'''
mol.basis = 'sto-3g'
mol.unit = 'Angstrom'
mol.build()

# RHF calculation
mf = scf.RHF(mol)
mf.kernel()
D = mf.make_rdm1()
h = mf.get_hcore()
eri = mol.intor('int2e')
J = np.einsum('rs,pqrs->pq', D, eri)
K = np.einsum('rs,prqs->pq', D, eri)

F = h + 0.5*(2 * J - K)
d, U = np.linalg.eigh(D)
E1 = 0.5 * np.trace(D@(h + F)) + mol.energy_nuc()
C = mf.mo_coeff

print(np.trace(D))
print(d)
print('|DF - FD|: ', np.trace(D@F - F@D))
print(mf.e_tot)

nocc = mf.mo_occ
print(C)
print(nocc)

i = 3
# D2 = U[:,i:i+1] @ np.diag(d[i:i+1]) @ U[:,i:i+1].T*2
davg = 0.5 * (d[2:3]+d[3:4])
D2 = U[:,i:i+1] @ np.diag(davg) @ U[:,i:i+1].T*2
D3 = (U[:,2:3]+U[:,3:4]) @ np.diag(davg) @ (U[:,2:3] + U[:,3:4]).T
print(U)
print('U[2:3]++ = ', (U[:,2:3]+U[:,3:4]).T)
print('d_avg = ', davg)
# D2 = U[:,2:4] @ np.diag(d[2:4]) @ U[:,2:4].T
# D2 = U[i:i+1, :].T @ np.diag(d[i:i+1]) @ U[i:i+1, :]*2
E2 = 0.5 * np.trace(D2@(h + F)) + mol.energy_nuc()
E3 = 0.5 * np.trace(D3@(h + F)) + mol.energy_nuc()
print('E1 = ', E1)
print('E2 = ', E2)
print('E3 = ', E3)
# 'Bosianise': Plug everything into lowest eigenvector
print("|E2 - E1| = ", np.linalg.norm(E2 - E1))
# Average between eigen-vectors & -values
print("|E3 - E1| = ", np.linalg.norm(E3 - E1))

# Confirm matching with PySCF Fock matrix
assert np.allclose(F, mf.get_fock()), "Mismatch in Fock matrix!"


converged SCF energy = -2.23351819973673
2.4098227442045874
[-5.71541280e-17 -7.79124758e-18  1.20491137e+00  1.20491137e+00]
|DF - FD|:  0.0
-2.2335181997367313
[[ 0.38808176  0.38808248  0.85733696 -0.8573362 ]
 [ 0.38809771  0.38809842 -0.85732975  0.85732899]
 [ 0.38809842 -0.38809771 -0.85732899 -0.85732975]
 [ 0.38808248 -0.38808176  0.8573362   0.85733696]]
[2. 2. 0. 0.]
[[-0.70680904 -0.02101237 -0.49998973 -0.49998973]
 [ 0.70678001  0.02101151 -0.50001027 -0.50001027]
 [-0.02101151  0.70678001 -0.50001027  0.50001027]
 [ 0.02101237 -0.70680904 -0.49998973  0.49998973]]
U[2:3]++ =  [[-9.99979461e-01 -1.00002054e+00  2.17091900e-10  1.41453810e-09]]
d_avg =  [1.20491137]
E1 =  -2.2335181997367313
E2 =  -2.2335181986919688
E3 =  -2.2335181997367304
|E2 - E1| =  1.044762498736418e-09
|E3 - E1| =  8.881784197001252e-16


In [4]:
"""
Find low-rank representation of D by minimizing tr((D-D_lr)*F) (i.e. that reproduces energy)

Parameters:
R: distance of H2s: H2 -----(R)----- H2
r: rank of rdm
"""

from pyscf import gto, scf
import numpy as np
import torch
from scipy.linalg import polar


def prepare_H4(R):
    mol = gto.Mole()
    mol.atom = f"""
    H 0 0 0
    H 0.74 0 0.
    H {R} 0 0
    H {R}.74 0 0
    """
    mol.basis = 'sto-3g'
    mol.unit = 'Angstrom'
    mol.build()

    # RHF calculation
    mf = scf.RHF(mol)
    mf.kernel()
    D = torch.tensor(mf.make_rdm1(), dtype=torch.float64)
    h = mf.get_hcore()
    eri = mol.intor('int2e')
    J = np.einsum('rs,pqrs->pq', D, eri)
    K = np.einsum('rs,prqs->pq', D, eri)
    enuc = mol.energy_nuc()

    F = h + 0.5*(2 * J - K)
    # print(f"Correct HF energy: {mf.e_tot}")
    return torch.tensor(h + F, dtype=torch.float64), D, mf, enuc, mol

def energy(D, F, enuc):
    return 0.5 * torch.trace(D@F) + enuc

def random_unitary(m, r):
    A = torch.randn(m, r, dtype=torch.float64)
    Q, _ = torch.linalg.qr(A)
    return Q

def buildD(U, d):
    return U @ torch.diag(d) @ U.T

def loss(U, d, D, F):
    D2 = buildD(U, d)
    return torch.norm(torch.trace((D - D2) @ F))

def polar_decomp (m, r):   # express polar decomposition in terms of singular-value decomposition
    U, S, Vh = torch.linalg.svd(m, full_matrices=False)
    u = U @ Vh
    # p = Vh.T.conj() @ S.diag().to (dtype = m.dtype) @ Vh
    return  u, None

def output(i, L0, L1, U, d, r, D, F):
    Dnew = buildD(U, d)
    e = energy(Dnew, F, enuc).item()
    err = torch.trace(Dnew @ F - F @ Dnew)
    print(i, L0.item(), L1.item(), err.item(), e - mf.e_tot)

def riemannian_steepest_descent(D, F, loss, r, h = 1e-4, iterations = 201, *args):
    U = random_unitary(F.shape[0], r)
    d = torch.abs(torch.randn(r, dtype=torch.float64, requires_grad=True))
    d *= nocc / torch.sum(d)
    # d /= torch.sum(d)
    for i in range(iterations):
        # gradients
        Utmp = U.detach().clone().requires_grad_()
        dtmp = d.detach().clone().requires_grad_()
        L0 = loss(Utmp, dtmp, D, F, *args)
        L0.backward(retain_graph=True)
        dU = Utmp.grad.detach()
        dd = dtmp.grad.detach()

        # step
        U_bk = U.clone()
        U = U - h * dU
        d = d - h * dd
        # d = d / torch.sum(d) * nocc

        U, _ = polar_decomp(U, r)

        L1 = loss(U, d, D, F, *args)

        err = torch.norm(L1 - L0)
        print(i, L0.item(), L1.item(), err.item())
        # output(i, L0, L1, U, d, r, D, F)

        if L1 < L0:
            h *= 1.5
        else:
            h /= 2.
            U = U_bk
            if h < 1e-14 or err < 1e-12:
                break
    return U, d

# Build two H3 molecules far apart (say 10 Angstroms separation)
R = 8
r = 4
F, D, mf, enuc, mol = prepare_H4(R)
nocc = torch.sum(torch.tensor(mf.mo_occ))
U, d = riemannian_steepest_descent(D, F, loss, r, 1e-4, 200)
print(U, d)

Dnew = U @ torch.diag(d) @ U.T
print(Dnew)
print(energy(Dnew, F, enuc))
def hf_energy(U, d, hcore, eri, mol, rdm, S):
    D = buildD(U, d)
    # replace with low-rank version for D if available
    J = torch.einsum('rs,pqrs->pq', rdm, eri)
    K = torch.einsum('rs,prqs->pq', rdm, eri)
    enuc = mol.energy_nuc()
    F = 2*hcore + 0.5*(2 * J - K)
    return 0.5 * torch.trace(D@F) + enuc


hcore = torch.tensor(mf.get_hcore())
eri = torch.tensor(mol.intor('int2e'))
S = torch.tensor(mf.get_ovlp())
Dnew = buildD(U, d)
print('E_HF(U, d, Dnew) = ', hf_energy(U, d, hcore, eri, mol, Dnew, S))
print('energy(Dnew) = ', energy(Dnew, F, enuc))

print('E_HF(U, d, D) = ', hf_energy(U, d, hcore, eri, mol, D, S))
print('energy(D) = ', energy(D, F, enuc))
"""
@TODO: Maybe this doesn't work because I'm only relaxing tr((D-Dnew)F) but F depends on D as well
it should be tr(Dnew F(Dnew)) = tr(D @ F(D))
"""

converged SCF energy = -2.23351786522334
0 2.825958486645749 2.822765248134696 0.0031932385110531314
1 2.822765248134696 2.817975356818974 0.004789891315721828
2 2.817975356818974 2.8107904505130534 0.007184906305920613
3 2.8107904505130534 2.800012955819843 0.010777494693210432
4 2.800012955819843 2.78384647955753 0.016166476262312823
5 2.78384647955753 2.759596474475902 0.024250005081628334
6 2.759596474475902 2.7232216096450075 0.03637486483089436
7 2.7232216096450075 2.6686623186791327 0.05455929096587475
8 2.6686623186791327 2.5868391808828552 0.08182313779627748
9 2.5868391808828552 2.4641703337143683 0.12266884716848692
10 2.4641703337143683 2.280416345845703 0.18375398786866537
11 2.280416345845703 2.005679177455707 0.2747371683899962
12 2.005679177455707 1.5966510700732082 0.4090281073824986
13 1.5966510700732082 0.9932485745302002 0.603402495543008
14 0.9932485745302002 0.11938117652459546 0.8738673980056048
15 0.11938117652459546 1.1059507884113422 0.9865696118867467
16 0.34

"\n@TODO: Maybe this doesn't work because I'm only relaxing tr((D-Dnew)F) but F depends on D as well\nit should be tr(Dnew F(Dnew)) = tr(D @ F(D))\n"

In [33]:
"""
Run SCF with low-rank density matrix

Parameters:
R: distance of H2s: H2 -----(R)----- H2
r: rank of rdm
"""

def prepare_system(R):
    mol = gto.Mole()
    mol.atom = f"""
    H 0 0 0
    H 0.74 0 0.
    H {R} 0 0
    H {R}.74 0 0
    # H {2*R} 0 0
    # H {2*R}.74 0 0
    """
    mol.basis = 'sto-3g'
    mol.unit = 'Angstrom'
    mol.build()

    # RHF calculation
    mf = scf.RHF(mol)
    mf.kernel()
    eri = torch.tensor(mol.intor('int2e'))
    print(f"Correct HF energy: {mf.e_tot}")
    return eri, mol, mf


def normalized_density(D, S, nocc):
    return nocc / 2 * D / torch.trace(D@S)


def purification_error(D, S, nocc):
    Dn = normalized_density(D, S, nocc)
    return torch.norm(Dn @ S @ Dn - Dn)


def hf_energy(U, d, hcore, eri, mol, rdm, S):
    D = buildD(U, d)
    # replace with low-rank version for D if available
    J = torch.einsum('rs,pqrs->pq', D, eri)
    K = torch.einsum('rs,prqs->pq', D, eri)
    enuc = mol.energy_nuc()
    F = 2*hcore + 0.5*(2 * J - K)
    return 0.5 * torch.trace(D@F) + enuc


def ev_purification(d, nocc, niter=10):
    d = d / torch.sum(d) * nocc / 2
    for _ in range(niter):
        d = 3 * d**2 - 2 * d**3
        d = d / torch.sum(d) * nocc / 2
    return d


def riemannian_steepest_descent(loss, S, r, nocc, hcore, h = 1e-4, iterations = 201, *args):
    # U = random_unitary(S.shape[0], r)
    # d = torch.abs(torch.randn(r, dtype=torch.float64))
    # d = ev_purification(d, nocc)

    """
    @TODO: try putting in correct guess and see if it stays there
    @TODO: How are d, U, and D different from method in cell above? 
            - Note: U seems coorect, so d must be wrong -> check this
    """

    d, U = torch.linalg.eigh(hcore)
    n = d.shape[0]
    d = d[n-r:]
    U = U[:, n-r:]
    d = ev_purification(d, nocc)
    print(U.shape, d.shape)
    print('U_start =\n', U)
    print('d_start = ', d)

    for i in range(iterations):
        Utmp = U.detach().clone().requires_grad_()
        dtmp = d.detach().clone().requires_grad_()
        L0 = loss(Utmp, dtmp, hcore, *args)
        L0.backward(retain_graph=True)
        dU = Utmp.grad.detach()
        dd = dtmp.grad.detach()

        U_bk = U.clone()
        U = U - h * dU
        d = d - h * dd

        U, _ = polar_decomp(U, r)
        d = ev_purification(d, nocc)
        d = d / torch.trace(buildD(U, d) @ S) * nocc
        # d = d / torch.trace(buildD(U, d) @ S) * nocc
        # d = d / torch.sum(d) * nocc

        L1 = loss(U, d, hcore, *args)

        err = torch.norm(L0 - L1)
        print(i, L0.item(), L1.item(), err.item(), d)

        if L1 < L0:
            h *= 1.5
        else:
            h /= 2.
            U = U_bk
            if h < 1e-12 or err < 1e-10:
                break
    return U, d


# Build two H3 molecules far apart (say 10 Angstroms separation)
R = 8
r = 1
eri, mol, mf = prepare_system(R)
hcore = torch.tensor(mf.get_hcore())
nocc = torch.sum(torch.tensor(mf.mo_occ))
S = torch.tensor(mf.get_ovlp())
D = torch.tensor(mf.make_rdm1())
Dn = normalized_density(D, S, nocc)

print('|D^2-D| = ', purification_error(D, S, nocc))

U, d = riemannian_steepest_descent(hf_energy, S, r, nocc, hcore, 1e-4, 150, eri, mol, D, S)
print('U_end =\n', U)
print('d_end = ', d)
print('E_end = ', hf_energy(U, d, hcore, eri, mol, D, S))
U = torch.tensor([[0.5, 0.5, 0.5, 0.5]], dtype=torch.float64).T
d = torch.tensor([2.2895], dtype=torch.float64)
print('E_manual = ', hf_energy(U, d, hcore, eri, mol, D, S))
print(S)

converged SCF energy = -2.23351786522334
Correct HF energy: -2.233517865223342
|D^2-D| =  tensor(2.2918e-16, dtype=torch.float64)
torch.Size([4, 1]) torch.Size([1])
U_start =
 tensor([[-0.5015],
        [ 0.4985],
        [-0.4985],
        [ 0.5015]], dtype=torch.float64)
d_start =  tensor([2.], dtype=torch.float64)
0 1.3263428252137537 0.7916201025134746 0.5347227227002791 tensor([11.7599], dtype=torch.float64)
1 0.7916201025134746 0.7916199974810614 1.050324132112479e-07 tensor([11.7599], dtype=torch.float64)
2 0.7916199974810614 0.7916198387345539 1.5874650749481134e-07 tensor([11.7599], dtype=torch.float64)
3 0.7916198387345539 0.7916195979029439 2.408316099877794e-07 tensor([11.7599], dtype=torch.float64)
4 0.7916195979029439 0.7916192304975076 3.674054362967638e-07 tensor([11.7599], dtype=torch.float64)
5 0.7916192304975076 0.7916186653416986 5.651558090402631e-07 tensor([11.7599], dtype=torch.float64)
6 0.7916186653416986 0.7916177853429158 8.799987827323719e-07 tensor([11.7600

In [78]:
import torch
from pyscf import gto, scf

# Build and solve molecule
mol = gto.M(atom='H 0 0 0; H 0 0 1; H 0 0 2; H 0 0 3', basis='sto-3g')
mf = scf.RHF(mol).run()
nocc = torch.sum(torch.tensor(mf.mo_occ))
print(nocc)

# Get density matrix and overlap
D = torch.tensor(mf.make_rdm1())
S = torch.tensor(mf.get_ovlp())

def normalized_density(D, S, nocc):
    return nocc / 2 * D / torch.trace(D@S)

def purification_error(D, S, nocc):
    Dn = normalized_density(D, S, nocc)
    return torch.norm(Dn @ S @ Dn - Dn)

# Measure deviation from idempotency
error = purification_error(D, S, nocc)

print(f"||DSD - D|| = {error.item():.3e}")


converged SCF energy = -2.09854593699772
tensor(4., dtype=torch.float64)
||DSD - D|| = 2.218e-16
